In [1]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
from modeler_strats.universal_modeler import UniversalModeler
from transformer.adhoc_transforms import Adhoc as adhocs
from transformer.risk_transforms import Risk as risks
from transformer.analysis_transforms import Analysis as anas

import numpy as np
import matplotlib.pyplot as plt
import pickle
from functional.backtester import Backtester as b
import math
from statistics import variance

In [2]:
start_date = datetime(2023,1,1)
end_date = datetime.now()

In [3]:
market = Market()
speculation_db = ADatabase("spec")
classification_db = ADatabase("spec_classification")
current_db = ADatabase("current")
umod = UniversalModeler()

In [4]:
market.connect()
sp500 = market.retrieve("sp500")
sp500 = sp500.rename(columns={"Symbol":"ticker"})
market.disconnect()

In [5]:
tyields = adhocs.tyields()
bench_returns = adhocs.spy_bench()

In [6]:
trades = []
values = [True,False]
classifications = [True,False]
ceilings = [True,False]
floors = [True,False]
parameters = []
positions = 10
strats = ["spec","rolling","window"]
for strat in strats:
    for value in values:
        for classification in classifications:
            for ceiling in ceilings:
                for floor in floors:
                    parameter = {
                                "value":value
                                 ,"classification":classification
                                 ,"ceiling":ceiling
                                 ,"floor":floor
                                ,"strat":strat
                                }
                    parameters.append(parameter)

In [7]:
market.connect()
sp500 = market.retrieve("sp500")
sp500 = sp500.rename(columns={"Symbol":"ticker"})
market.disconnect()

In [14]:
speculation_db.connect()
simulation = speculation_db.retrieve("predictions")
speculation_db.disconnect()

In [15]:
classification_db.connect()
classification_simulation = classification_db.retrieve("predictions")
classification_db.disconnect()

In [16]:
new_prices = []
market.connect()
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        ticker_sim = market.retrieve_ticker_prices("prices",ticker)
        completed = risks.weekly_risk_prep(ticker_sim,bench_returns)
        new_prices.append(completed)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
price_returns = pd.concat(new_prices)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:49<00:00,  9.97it/s]


In [17]:
simulation

,year,week,ticker,adjclose,0,1,2,3,4,5,...,d2,d3,rolling14,xgb_prediction,xgb_score,cat_prediction,cat_score,tf_prediction,tf_score,prediction
0,2023,2,MMM,122.681938,122.681938,118.167401,120.018608,123.209868,123.805885,124.279542,...,-3.476904,7.595792,119.935581,123.393387,0.99911,123.298255,0.937924,123.487595,0.996624,123.393079
1,2023,3,MMM,126.880705,126.880705,122.681938,118.167401,120.018608,123.209868,123.805885,...,-0.104170,-0.970039,121.158728,127.931686,0.99911,126.306590,0.937924,127.037865,0.996624,127.092047
2,2023,4,MMM,120.493744,120.493744,126.880705,122.681938,118.167401,120.018608,123.209868,...,-2.470813,22.719010,122.044239,121.359016,0.99911,120.771690,0.937924,125.509865,0.996624,122.546857
3,2023,5,MMM,114.338678,114.338678,120.493744,126.880705,122.681938,118.167401,120.018608,...,0.014775,-1.005980,122.214356,114.323181,0.99911,115.482854,0.937924,121.736809,0.996624,117.180948
4,2023,6,MMM,114.723524,114.723524,114.338678,120.493744,126.880705,122.681938,118.167401,...,-1.065891,-73.143548,121.923588,114.323181,0.99911,114.572088,0.937924,119.875771,0.996624,116.257013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,2023,18,ZTS,174.408000,174.408000,174.841118,171.964237,167.469901,164.446403,163.420608,...,-1.148074,2.048386,167.721723,173.420776,0.99911,175.486834,0.937924,178.244064,0.996624,175.717225
10242,2023,19,ZTS,179.778000,179.778000,174.408000,174.841118,171.964237,167.469901,164.446403,...,-13.429258,10.697206,168.708926,178.915390,0.99911,181.177957,0.937924,182.182175,0.996624,180.758507
10243,2023,20,ZTS,184.624000,184.624000,179.778000,174.408000,174.841118,171.964237,167.469901,...,-0.124535,-0.990727,169.967718,184.204208,0.99911,186.102953,0.937924,186.328445,0.996624,185.545202
10244,2023,21,ZTS,180.702000,180.702000,184.624000,179.778000,174.408000,174.841118,171.964237,...,-1.788084,13.358130,171.304027,181.405548,0.99911,181.177957,0.937924,185.255722,0.996624,182.613076


In [18]:
sim = price_returns.merge(simulation[["year","week","ticker","prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(classification_simulation[["year","week","ticker","classification_prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(tyields[["date","weekly_yield"]],on="date",how="left")


In [19]:
sim.sort_values(["date","ticker"],inplace=True)
sim = sim.groupby(["date","ticker","classification_prediction"]).mean().reset_index()

In [20]:
current_db.connect()
current_db.drop("trades")
for parameter in tqdm(parameters):
    backtest_data = sim.copy().dropna()
    strat = parameter["strat"]
    backtest_data = risks.weekly_backtest_prep(strat,backtest_data)
    b.weeklies_backtest(backtest_data.copy(),positions,parameter,start_date,end_date,current_db)
current_db.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [01:11<00:00,  1.49s/it]
